In [1]:
#! pip install python-resize-image

In [2]:
#! pip install tensorflow

In [3]:
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D, Activation, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory

import tensorflow as tf
from tensorflow.keras.models import model_from_json
import cv2
#from skimage import color
from PIL import Image
#import imageio
#from resizeimage import resizeimage
import os

import pickle

from sklearn.metrics import confusion_matrix

In [4]:
#drive.mount('/content/drive')

In [5]:
#files.upload()

In [6]:
#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json

In [7]:
#!kaggle datasets download -d validmodel/indo-fashion-dataset

In [8]:
#!unzip -q /content/indo-fashion-dataset.zip


In [9]:
df_train = pd.read_json('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/data/raw/indian_clothing/train_data.json', lines=True)

In [10]:
df_train.head()

,image_url,image_path,brand,product_title,class_label,color
0,https://m.media-amazon.com/images/I/81XKaSKvly...,images/train/0.jpeg,Womanista,Women's Georgette Saree with Blouse Piece (TKI...,saree,NaN
1,https://m.media-amazon.com/images/I/91dDMDg+QT...,images/train/1.jpeg,Yashika,Women Linen Printed Sarees Free Size (BEE),saree,NaN
2,https://m.media-amazon.com/images/I/71TJ9p46PV...,images/train/2.jpeg,Womanista,Organza Floral Print Saree with Blouse Piece(W...,saree,NaN
3,https://m.media-amazon.com/images/I/81Y+je7CEg...,images/train/3.jpeg,PERFECTBLUE,Women's Digital Cotton Linen Blend Saree with ...,saree,NaN
4,https://m.media-amazon.com/images/I/61B8o9Ulqp...,images/train/4.jpeg,VAIVIDHYAM,Designer Sarees Women's Banarasi Cotton Silk S...,saree,NaN


In [11]:
df_train.drop(['image_url', 'brand', 'product_title', 'color'], axis=1, inplace=True)

In [12]:
df_train.head()

,image_path,class_label
0,images/train/0.jpeg,saree
1,images/train/1.jpeg,saree
2,images/train/2.jpeg,saree
3,images/train/3.jpeg,saree
4,images/train/4.jpeg,saree


In [13]:
df_test = pd.read_json('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/data/raw/indian_clothing/test_data.json', lines = True)

In [14]:
df_test.head()

,image_url,image_path,brand,product_title,class_label,color
0,https://m.media-amazon.com/images/I/71NZQ-aaAS...,images/test/0.jpeg,TANTUJA BENGAL HANDLOOM,(A Govt. of West Bengal Enterprises) Handloom ...,saree,NaN
1,https://m.media-amazon.com/images/I/81aQmbWsG7...,images/test/1.jpeg,Trilok Fab,Women's Silk Saree With Blouse Piece,saree,NaN
2,https://m.media-amazon.com/images/I/71YjGvzs9y...,images/test/2.jpeg,Nine Sister,Women's Georgette Net Embroidered Saree with B...,saree,NaN
3,https://m.media-amazon.com/images/I/81JYhYV5Xr...,images/test/3.jpeg,Gauri Laxmi Enterprise,Women's Crepe Saree with Blouse (GLE$CHICKOO C...,saree,NaN
4,https://m.media-amazon.com/images/I/91fXc5i5Dy...,images/test/4.jpeg,SOURBH,Women's Cotton Blend Digital Butta Printed Sar...,saree,NaN


In [15]:
df_test.drop(['image_url', 'brand', 'product_title', 'color'], axis=1, inplace=True)

In [16]:
df_test.tail()

,image_path,class_label
7495,images/test/7495.jpeg,women_kurta
7496,images/test/7496.jpeg,women_kurta
7497,images/test/7497.jpeg,women_kurta
7498,images/test/7498.jpeg,women_kurta
7499,images/test/7499.jpeg,women_kurta


In [17]:
#df_test.to_csv('/content/drive/My Drive/capstone_3/df_test.csv')

In [18]:

df_val = pd.read_json('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/data/raw/indian_clothing/val_data.json', lines=True)

In [19]:
df_val.head()

,image_url,image_path,brand,product_title,class_label,color
0,https://m.media-amazon.com/images/I/81LOPbFPiQ...,images/val/0.jpeg,Generic,Women's Khadi Cotton Saree With Blouse Piece (...,saree,NaN
1,https://m.media-amazon.com/images/I/81Q8Oktw4s...,images/val/1.jpeg,Yashvi Designer,Women's Net Saree With Unstitched Blouse Piece,saree,NaN
2,https://m.media-amazon.com/images/I/613S1YGCmo...,images/val/2.jpeg,Aarrah,Georgette Strip Print Saree[S_SHIKHA30021SR02_...,saree,NaN
3,https://m.media-amazon.com/images/I/819budhQl1...,images/val/3.jpeg,MIMOSA,Women's Patola Style Art Silk Saree (Green),saree,NaN
4,https://m.media-amazon.com/images/I/61cAZ94ZQV...,images/val/4.jpeg,Generic,Women's Pure Hand Block Patola Printed Cotton ...,saree,NaN


In [20]:

df_val.drop(['image_url', 'brand', 'product_title', 'color'], axis=1, inplace=True)

In [21]:
df_val.head()

,image_path,class_label
0,images/val/0.jpeg,saree
1,images/val/1.jpeg,saree
2,images/val/2.jpeg,saree
3,images/val/3.jpeg,saree
4,images/val/4.jpeg,saree


In [22]:
df_train = pd.get_dummies(data=df_train, columns=['class_label'])

In [23]:
df_train.head()

,image_path,class_label_blouse,class_label_dhoti_pants,class_label_dupattas,class_label_gowns,class_label_kurta_men,class_label_leggings_and_salwars,class_label_lehenga,class_label_mojaris_men,class_label_mojaris_women,class_label_nehru_jackets,class_label_palazzos,class_label_petticoats,class_label_saree,class_label_sherwanis,class_label_women_kurta
0,images/train/0.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,images/train/1.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,images/train/2.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,images/train/3.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,images/train/4.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [24]:
df_val = pd.get_dummies(data=df_val, columns=['class_label'])

In [25]:
df_val.head()

,image_path,class_label_blouse,class_label_dhoti_pants,class_label_dupattas,class_label_gowns,class_label_kurta_men,class_label_leggings_and_salwars,class_label_lehenga,class_label_mojaris_men,class_label_mojaris_women,class_label_nehru_jackets,class_label_palazzos,class_label_petticoats,class_label_saree,class_label_sherwanis,class_label_women_kurta
0,images/val/0.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,images/val/1.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,images/val/2.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,images/val/3.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,images/val/4.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [26]:
df_test = pd.get_dummies(data=df_test, columns=['class_label'])

In [27]:
df_train.columns

Index(['image_path', 'class_label_blouse', 'class_label_dhoti_pants',
       'class_label_dupattas', 'class_label_gowns', 'class_label_kurta_men',
       'class_label_leggings_and_salwars', 'class_label_lehenga',
       'class_label_mojaris_men', 'class_label_mojaris_women',
       'class_label_nehru_jackets', 'class_label_palazzos',
       'class_label_petticoats', 'class_label_saree', 'class_label_sherwanis',
       'class_label_women_kurta'],
      dtype='object')

In [28]:
# Creating generator images sizes 64 x 64
columns = ['class_label_blouse', 'class_label_dhoti_pants',
       'class_label_dupattas', 'class_label_gowns', 'class_label_kurta_men',
       'class_label_leggings_and_salwars', 'class_label_lehenga',
       'class_label_mojaris_men', 'class_label_mojaris_women',
       'class_label_nehru_jackets', 'class_label_palazzos',
       'class_label_petticoats', 'class_label_saree', 'class_label_sherwanis',
       'class_label_women_kurta']
train_datagen=ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(dataframe=df_train, directory='/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/data/raw/indian_clothing/', x_col='image_path', y_col=columns, target_size=(32,32), batch_size=128, seed=26, save_to_dir='/Users/butler/Documents/untitled/', class_mode='raw')

Found 91166 validated image filenames.


In [29]:
columns = ['class_label_blouse', 'class_label_dhoti_pants',
       'class_label_dupattas', 'class_label_gowns', 'class_label_kurta_men',
       'class_label_leggings_and_salwars', 'class_label_lehenga',
       'class_label_mojaris_men', 'class_label_mojaris_women',
       'class_label_nehru_jackets', 'class_label_palazzos',
       'class_label_petticoats', 'class_label_saree', 'class_label_sherwanis',
       'class_label_women_kurta']
val_datagen=ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(dataframe=df_val, directory='/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/data/raw/indian_clothing/', x_col='image_path', y_col=columns, target_size=(32,32), batch_size=128, seed=26, save_to_dir='/Users/butler/Documents/untitled/', class_mode='raw')

Found 7500 validated image filenames.


In [28]:
columns = ['class_label_blouse', 'class_label_dhoti_pants',
       'class_label_dupattas', 'class_label_gowns', 'class_label_kurta_men',
       'class_label_leggings_and_salwars', 'class_label_lehenga',
       'class_label_mojaris_men', 'class_label_mojaris_women',
       'class_label_nehru_jackets', 'class_label_palazzos',
       'class_label_petticoats', 'class_label_saree', 'class_label_sherwanis',
       'class_label_women_kurta']
test_datagen=ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(dataframe=df_test, directory='/content/', x_col='image_path', y_col=columns, target_size=(32,32), batch_size=128, seed=26, save_to_dir='/Users/butler/Documents/untitled2/', class_mode='raw')

Found 0 validated image filenames.


/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 7500 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(


In [32]:
# Setting up model
img_width, img_height = 32, 32

train_data = train_generator
val_data = val_generator
nb_train_samples = 91166
nb_validation_samples = 7500
epochs = 500
batch_size = 64
n_classes = 15

input_shape = (img_width, img_height, 3)

In [37]:

model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3,3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models',
        save_best_only=True,
        monitor='val_accuracy'
    )
]

In [41]:
history = model.fit(
    x=train_data,
    y=None,
    validation_data= val_data,
    epochs=epochs,
    verbose="auto",
    callbacks=callbacks,
    validation_split=0.0,
    shuffle=True,
    validation_freq=10,
    )

Epoch 1/500


2023-08-31 17:05:46.738091: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


713/713 [==============================] - 119s 166ms/step - loss: 1.1546 - accuracy: 0.6263
Epoch 2/500
713/713 [==============================] - 129s 180ms/step - loss: 1.0283 - accuracy: 0.6672
Epoch 3/500
713/713 [==============================] - 130s 182ms/step - loss: 0.9492 - accuracy: 0.6928
Epoch 4/500
713/713 [==============================] - 129s 180ms/step - loss: 0.8887 - accuracy: 0.7124
Epoch 5/500
713/713 [==============================] - 138s 194ms/step - loss: 0.8444 - accuracy: 0.7264
Epoch 6/500
713/713 [==============================] - 137s 192ms/step - loss: 0.8124 - accuracy: 0.7354
Epoch 7/500
713/713 [==============================] - 131s 183ms/step - loss: 0.7752 - accuracy: 0.7493
Epoch 8/500
713/713 [==============================] - 139s 195ms/step - loss: 0.7529 - accuracy: 0.7547
Epoch 9/500
713/713 [==============================] - 143s 200ms/step - loss: 0.7253 - accuracy: 0.7637
Epoch 10/500
713/713 [==============================] - ETA: 0s - l

2023-08-31 17:28:05.507957: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-08-31 17:28:17.087197: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32]
	 [[{{node inputs}}]]
2023-08-31 17:28:17.189179: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/assets


...

713/713 [==============================] - 167s 235ms/step - loss: 0.4743 - accuracy: 0.8347 - val_loss: 0.7986 - val_accuracy: 0.7805
Epoch 31/500
713/713 [==============================] - ETA: 0s - loss: 0.4740 - accuracy: 0.8361WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


713/713 [==============================] - 156s 219ms/step - loss: 0.4740 - accuracy: 0.8361
Epoch 32/500
713/713 [==============================] - ETA: 0s - loss: 0.4644 - accuracy: 0.8393WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


713/713 [==============================] - 152s 213ms/step - loss: 0.4644 - accuracy: 0.8393
Epoch 33/500
713/713 [==============================] - ETA: 0s - loss: 0.4601 - accuracy: 0.8384WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


713/713 [==============================] - 149s 209ms/step - loss: 0.4601 - accuracy: 0.8384
Epoch 34/500
451/713 [=================>............] - ETA: 1:50 - loss: 0.4501 - accuracy: 0.8426

KeyboardInterrupt: 

In [46]:
scores = model.evaluate(x=val_generator, y=None, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

2023-08-31 18:31:25.501599: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


KeyboardInterrupt: 

In [47]:
model.save('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model_11.keras')

In [45]:
tf.keras.saving.save_model(
    model, '/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model_11a', overwrite=True, save_format=None)

2023-08-31 18:31:00.579246: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32]
	 [[{{node inputs}}]]
2023-08-31 18:31:00.675646: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model_11a/assets


INFO:tensorflow:Assets written to: /Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model_11a/assets


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices()

In [29]:
loaded_model = tf.keras.models.load_model('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models')

In [30]:
loaded_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_8 (Conv2D)           (None, 10, 10, 64)        36928     
                                                                 
 conv2d_9 (Conv2D)           (None, 8, 8, 64)          36928     
                                                      

In [50]:
!!loaded_model.load_weights('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models')

2023-08-31 18:37:41.919785: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open /Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models: FAILED_PRECONDITION: /Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [ ]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
history = loaded_model.fit(
    x=train_data,
    y=None,
    validation_data= val_data,
    epochs=epochs,
    verbose="auto",
    callbacks=callbacks,
    validation_split=0.0,
    shuffle=True,
    validation_freq=10,
    )

Epoch 1/500


2023-08-31 18:44:36.166214: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-08-31 18:44:36.186251: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


713/713 [==============================] - 156s 218ms/step - loss: 0.4746 - accuracy: 0.8359
Epoch 2/500
713/713 [==============================] - 157s 221ms/step - loss: 0.4615 - accuracy: 0.8388
Epoch 3/500
713/713 [==============================] - 145s 204ms/step - loss: 0.4554 - accuracy: 0.8406
Epoch 4/500
713/713 [==============================] - 149s 208ms/step - loss: 0.4547 - accuracy: 0.8406
Epoch 5/500
713/713 [==============================] - 158s 221ms/step - loss: 0.4506 - accuracy: 0.8434
Epoch 6/500
713/713 [==============================] - 150s 210ms/step - loss: 0.4430 - accuracy: 0.8432
Epoch 7/500
713/713 [==============================] - 159s 223ms/step - loss: 0.4397 - accuracy: 0.8472
Epoch 8/500
713/713 [==============================] - 164s 230ms/step - loss: 0.4361 - accuracy: 0.8470
Epoch 9/500
713/713 [==============================] - 173s 241ms/step - loss: 0.4379 - accuracy: 0.8466
Epoch 10/500
713/713 [==============================] - ETA: 0s - l

2023-08-31 19:11:11.187438: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-08-31 19:11:22.690636: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32]
	 [[{{node inputs}}]]
2023-08-31 19:11:22.819782: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/assets


...

713/713 [==============================] - 172s 241ms/step - loss: 0.2503 - accuracy: 0.9064
Epoch 122/500
713/713 [==============================] - ETA: 0s - loss: 0.2477 - accuracy: 0.9067WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


713/713 [==============================] - 182s 254ms/step - loss: 0.2477 - accuracy: 0.9067
Epoch 123/500
713/713 [==============================] - ETA: 0s - loss: 0.2474 - accuracy: 0.9077WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


713/713 [==============================] - 190s 265ms/step - loss: 0.2474 - accuracy: 0.9077
Epoch 124/500
453/713 [==================>...........] - ETA: 1:10 - loss: 0.2382 - accuracy: 0.9089

2023-09-01 01:00:06.728123: W tensorflow/core/framework/op_kernel.cc:1818] UNKNOWN: OSError: [Errno 28] No space left on device
Traceback (most recent call last):

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/Image.py", line 2431, in save
    save_handler(self, fp, filename)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/PngImagePlugin.py", line 1434, in _save
    fp.flush()

OSError: [Errno 28] No space left on device


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/tensorflow/python/data/ops/

UnknownError: Graph execution error:

OSError: [Errno 28] No space left on device
Traceback (most recent call last):

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/Image.py", line 2431, in save
    save_handler(self, fp, filename)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/PngImagePlugin.py", line 1434, in _save
    fp.flush()

OSError: [Errno 28] No space left on device


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_

In [37]:
scores = loaded_model.evaluate(x=val_generator, y=None, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], scores[1]*100))

2023-09-01 06:27:26.552536: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


accuracy: 75.67%


In [38]:
loaded_model.save('model.keras')

In [41]:
#!pip install h5py
# serialize model to JSON
loaded_model_json = loaded_model.to_json()
with open('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model20230831.json', 'w') as json_file:
    json_file.write(loaded_model_json)
# serialize weights to HDF5
loaded_model.save_weights('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model20230831.h5')

In [30]:
json_file = open('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model20230831.json', 'r')
loaded_model2_json = json_file.read()
json_file.close()
loaded_model2 = model_from_json(loaded_model2_json)

In [31]:
loaded_model2.load_weights('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model20230831.h5')
print("Loaded model from disk")

Loaded model from disk


In [32]:
# Setting up model
img_width, img_height = 32, 32

train_data = train_generator
val_data = val_generator
nb_train_samples = 91166
nb_validation_samples = 7500
epochs = 500
batch_size = 64
n_classes = 15

input_shape = (img_width, img_height, 3)

In [33]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model',
        save_best_only=True,
        monitor='val_accuracy'
    )
]

In [34]:
loaded_model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
history = loaded_model2.fit(
    x=train_data,
    y=None,
    validation_data= val_data,
    epochs=epochs,
    verbose="auto",
    callbacks=callbacks,
    validation_split=0.0,
    shuffle=True,
    validation_freq=10,
    )

Epoch 1/500
713/713 [==============================] - 193s 270ms/step - loss: 0.2459 - accuracy: 0.9092
Epoch 2/500
713/713 [==============================] - 174s 244ms/step - loss: 0.2429 - accuracy: 0.9095
Epoch 3/500
713/713 [==============================] - 164s 230ms/step - loss: 0.2412 - accuracy: 0.9102
Epoch 4/500
713/713 [==============================] - 173s 243ms/step - loss: 0.2483 - accuracy: 0.9062
Epoch 5/500
713/713 [==============================] - 206s 289ms/step - loss: 0.2415 - accuracy: 0.9087
Epoch 6/500
713/713 [==============================] - 213s 299ms/step - loss: 0.2465 - accuracy: 0.9073
Epoch 7/500
713/713 [==============================] - 182s 255ms/step - loss: 0.2423 - accuracy: 0.9082
Epoch 8/500
713/713 [==============================] - 200s 280ms/step - loss: 0.2397 - accuracy: 0.9090
Epoch 9/500
713/713 [==============================] - 205s 287ms/step - loss: 0.2428 - accuracy: 0.9096
Epoch 10/500
713/713 [==============================] -

INFO:tensorflow:Assets written to: /Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model/assets


713/713 [==============================] - 207s 290ms/step - loss: 0.2477 - accuracy: 0.9068 - val_loss: 1.6138 - val_accuracy: 0.7547
Epoch 11/500
713/713 [==============================] - ETA: 0s - loss: 0.2393 - accuracy: 0.9098WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


713/713 [==============================] - 194s 272ms/step - loss: 0.2393 - accuracy: 0.9098
Epoch 12/500
713/713 [==============================] - ETA: 0s - loss: 0.2385 - accuracy: 0.9102WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


713/713 [==============================] - 202s 283ms/step - loss: 0.2385 - accuracy: 0.9102
Epoch 13/500
713/713 [==============================] - ETA: 0s - loss: 0.2403 - accuracy: 0.9089WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


...

713/713 [==============================] - 176s 244ms/step - loss: 0.2072 - accuracy: 0.9227
Epoch 118/500
713/713 [==============================] - ETA: 0s - loss: 0.2021 - accuracy: 0.9242WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


713/713 [==============================] - 178s 247ms/step - loss: 0.2021 - accuracy: 0.9242
Epoch 119/500
713/713 [==============================] - ETA: 0s - loss: 0.2076 - accuracy: 0.9209WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


713/713 [==============================] - 183s 256ms/step - loss: 0.2076 - accuracy: 0.9209
Epoch 120/500
339/713 [=============>................] - ETA: 1:36 - loss: 0.2015 - accuracy: 0.9242

2023-09-01 16:24:35.942304: W tensorflow/core/framework/op_kernel.cc:1816] UNKNOWN: OSError: [Errno 28] No space left on device
Traceback (most recent call last):

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/Image.py", line 2431, in save
    save_handler(self, fp, filename)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/PngImagePlugin.py", line 1434, in _save
    fp.flush()

OSError: [Errno 28] No space left on device


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/tensorflow/python/data/ops/

UnknownError: Graph execution error:

OSError: [Errno 28] No space left on device
Traceback (most recent call last):

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/Image.py", line 2431, in save
    save_handler(self, fp, filename)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/PngImagePlugin.py", line 1434, in _save
    fp.flush()

OSError: [Errno 28] No space left on device


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_

In [36]:
loaded_model2_json = loaded_model2.to_json()
with open('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model20230831_2.json', 'w') as json_file:
    json_file.write(loaded_model2_json)
# serialize weights to HDF5
loaded_model2.save_weights('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model20230831_2.h5')

In [34]:
json_file = open('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model20230831_2.json', 'r')
loaded_model3_json = json_file.read()
json_file.close()
loaded_model3 = model_from_json(loaded_model3_json)

In [35]:
loaded_model3.load_weights('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model20230831_2.h5')
print("Loaded model from disk")

Loaded model from disk


In [37]:
loaded_model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
scores = loaded_model3.evaluate(x=val_generator, y=None, verbose=0)
print("%s: %.2f%%" % (loaded_model3.metrics_names[1], scores[1]*100))

accuracy: 75.55%
